In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

url = 'https://mcdindia.com/'
service = Service(r"C:\Program Files (x86)\chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.get(url)
driver.maximize_window()
time.sleep(12)

menu_items = driver.find_elements(By.XPATH, "(//ul/li/a[@class ='menu-item-link js-smooth-scroll'])[1]")

if menu_items:
    menu_items[0].click()
    time.sleep(3)

data = {}

sections = driver.find_elements(By.XPATH, '//div[contains(@class, "menu-slider-navigation")]/div[contains(@class, "main-heading")]')
j = 2
nav = 2
element_num = 1
box = 16
box_2 = 24


columns = set()
rows = []

for i in sections:
    if j > 2:
        nav += 1
        element_num += 1
        driver.execute_script("window.scrollTo(0,-500);")
        i.click()
        driver.execute_script("window.scrollTo(0,1000);")
        items = driver.find_elements(By.XPATH, f'(//div[@class="slides-inner"])[{element_num}]//div[@class="para-text"]')

        for item in items:
            item_name = item.text
            if item_name not in data.values():
                data[item_name] = item_name  
                if element_num < 3:  # For first two sections
                    sub_texts = driver.find_elements(By.XPATH, f'(//div//div[@class="slide-nutrition"])[{box}]//p[@class="sub-text"]')
                    values = driver.find_elements(By.XPATH, f'(//div//div[@class="slide-nutrition"])[{box}]//p[@class="value-text value-1 active"]')
                    row = {'Item': item_name}
                    for sub_text, value_text in zip(sub_texts, values):
                        columns.add(sub_text.text)
                        row[sub_text.text] = value_text.text
                    rows.append(row)
                    box += 1
                else:  # Starting from the third section
                    sub_texts = driver.find_elements(By.XPATH, f'(//div//div[@class="slide-nutrition"])[{box_2}]//p[@class="sub-text"]')
                    values = driver.find_elements(By.XPATH, f'(//div//div[@class="slide-nutrition"])[{box_2}]//p[@class="value-text value-1 active"]')
                    row = {'Item': item_name}
                    for index, (sub_text, value_text) in enumerate(zip(sub_texts, values), start=24):
                        columns.add(sub_text.text)
                        row[sub_text.text] = value_text.text
                    rows.append(row)
                    box_2 += 1

                next_button = driver.find_element(By.XPATH, f'(//div//div[@class="slide-nav-arrows"])[{nav}]//div[2]')
                next_button.click()
                time.sleep(2)
            else:
                print("Duplicate names found")
                break

    else:
        items = driver.find_elements(By.XPATH, '(//div[@class="slides-inner"])[1]//div[@class="para-text"]')
        box = 1
        
        for item in items:
            item_name = item.text
            if item_name not in data.values():
                data[item_name] = item_name  # Use item_name as a unique key temporarily
                sub_texts = driver.find_elements(By.XPATH, f'(//div//div[@class="slide-nutrition"])[{box}]//p[@class="sub-text"]')
                values = driver.find_elements(By.XPATH, f'(//div//div[@class="slide-nutrition"])[{box}]//p[@class="value-text value-1 active"]')
                row = {'Item': item_name}
                for sub_text, value_text in zip(sub_texts, values):
                    columns.add(sub_text.text)
                    row[sub_text.text] = value_text.text
                rows.append(row)
                
                next_button = driver.find_element(By.XPATH, '(//div//div[@class="slide-nav-arrows"])[2]//div[2]')
                next_button.click()
                time.sleep(2)
                box += 1
            else:
                print("Duplicate names found")
                break

        j += 1


df = pd.DataFrame(rows, columns=['Item'] + list(columns))


output_file_path = r"C:\Users\Lipika\Downloads\MCD\MCD 2.xlsx"
df.to_excel(output_file_path, index=False)

print(f'DataFrame saved to {output_file_path}')


driver.quit()


Duplicate names found
DataFrame saved to C:\Users\Lipika\Downloads\MCD\MCD 2.xlsx
